<a href="https://colab.research.google.com/github/custom-hyper/CoinGeckoAPI/blob/main/CoinGecko_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title Install Libraries { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install pycoingecko

  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=03159b4b543e37baf55909d598604d66ae634f0c277c974479848e1546b7070b
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


In [ ]:
#@title Available Coins { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

#General Information table

def coins_stats(url):
    """
    Generates a list of the coins on CoinGecko
    Args: url of the CoinGecko website

    """
    frames = []
    response = requests.get(url)
    data = response.json()

    
    token_list = pd.DataFrame(data)

    print('Stats: ')
    print(token_list.info(verbose=True))
    print(token_list.isnull().sum())
    print(token_list.describe())
   
    return token_list 


def main():
  url = "https://api.coingecko.com/api/v3/coins/list"

  coins_stats(url)
  

if __name__ == "__main__":
    main()

Stats: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13491 entries, 0 to 13490
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      13491 non-null  object
 1   symbol  13491 non-null  object
 2   name    13491 non-null  object
dtypes: object(3)
memory usage: 316.3+ KB
None
id        0
symbol    0
name      0
dtype: int64
            id symbol    name
count    13491  13491   13491
unique   13491  11217   13491
top     01coin   moon  01coin
freq         1     10       1


# Close Prices

In [ ]:
#@title  Bulk Download { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import logging
import sys
from tqdm.notebook import tqdm

#Extract token list
url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)

 #suppressing carriers
id = token_list.id
currency = 'usd'
Horizon = 'max'

#loop  prices extraction to token list
try:

  start = 0
  ends =   100

  for i in id[start : ends]:

    response = requests.get("https://api.coingecko.com/api/v3/coins/{}/market_chart?vs_currency={}&days={}".format(i,currency,Horizon))
    output = response.text
    json_parsed = json.loads(output)
    print(json_parsed)

    #Convert data to list of dataframes
    full_table = pd.json_normalize(json_parsed)       
    full_table['id'] = i
    #full_table.loc(i)
    frames.append(full_table)

    #Slow down requestsloc

    time.sleep(60/50)
    #Status report
    logging.info('Page '+ str(i) + ' – Rows ' + ' – source: '+ str(url))
except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop
df = single

print(single)
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

single.to_csv( path + 'CoinGecko_price_list.csv')




{'prices': [[1536969600000, 0.009788403063705275], [1537056000000, 0.009882558743828542], [1537142400000, 0.012145158274011318], [1537228800000, 0.007965204513941592], [1537315200000, 0.011866053006407695], [1537401600000, 0.011457442133231787], [1537488000000, 0.009856303171318184], [1537574400000, 0.010346361715446901], [1537660800000, 0.012509513238031308], [1537747200000, 0.00872956465326417], [1537833600000, 0.01005844860946907], [1537920000000, 0.008786264540711866], [1538006400000, 0.008104729360386247], [1538092800000, 0.00929708644129806], [1538179200000, 0.008332842978249768], [1538265600000, 0.00710890526630388], [1538352000000, 0.00704758470712929], [1538438400000, 0.010964302136757384], [1538524800000, 0.00783325848098873], [1538611200000, 0.00727669373334877], [1538697600000, 0.007948086657450218], [1538784000000, 0.010100972410891288], [1538870400000, 0.012831612472876296], [1538956800000, 0.011294693135382498], [1539043200000, 0.013286846597662993], [1539129600000, 0.01

100

 # OHLC Prices 


Candlestick  body:

1 - 2 days: 30 minutes
3 - 30 days: 4 hours
31 and before: 4 days

In [17]:
#@title Bulk Download { display-mode: "code" }

from google.colab import data_table
from pycoingecko import CoinGeckoAPI
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from __future__ import division
import requests
import datetime
import time
import json

#Extract token list
url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)

cg = CoinGeckoAPI()
 #suppressing carriers
id = token_list.id
currency = 'usd'
Horizon = 'max'

#loop  prices extraction to token list


start = 0
ends =   100
frames = []
try:
  for i in id[start : ends]:
  #Chart
    a = cg.get_coin_ohlc_by_id(id=i, vs_currency=currency, days = Horizon)

    df = pd.DataFrame(data=a, columns = ['Date','Open','High', 'Low', 'Close'])

    
    df['id'] = i
      #full_table.loc(i)
    frames.append(df)
    print(df)
      #Slow down requestsloc

    time.sleep(60/50)
      #Status report

except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop

    
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

single.to_csv( path + 'CoinGecko_ohlc_timeseries.csv')


              Date      Open      High       Low     Close      id
0    1536969600000  0.009788  0.009788  0.009788  0.009788  01coin
1    1537315200000  0.009883  0.012145  0.007965  0.011866  01coin
2    1537660800000  0.011457  0.012510  0.009856  0.012510  01coin
3    1538006400000  0.008730  0.010058  0.008105  0.008105  01coin
4    1538265600000  0.009297  0.009297  0.007109  0.007109  01coin
..             ...       ...       ...       ...       ...     ...
334  1647648000000  0.000512  0.000615  0.000512  0.000544  01coin
335  1647993600000  0.000420  0.000536  0.000420  0.000509  01coin
336  1648339200000  0.000443  0.000618  0.000443  0.000618  01coin
337  1648684800000  0.000468  0.000817  0.000468  0.000817  01coin
338  1648944000000  0.000706  0.000706  0.000463  0.000466  01coin

[339 rows x 6 columns]
              Date      Open      High       Low     Close  \
0    1597795200000  11007.43  11007.43  11007.43  11007.43   
1    1598140800000  10946.39  11086.45  10946.39

KeyboardInterrupt: ignored

In [ ]:
#@title Generate  Charts { display-mode: "code" }

from google.colab import data_table
from pycoingecko import CoinGeckoAPI
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from __future__ import division


import requests
import datetime
import time
import json

#Extract token list
url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)

cg = CoinGeckoAPI()
 #suppressing carriers
id = token_list.id
currency = 'usd'
Horizon = 'max'

#loop  prices extraction to token list


start = 0
ends =   100

for i in id[start : ends]:
#Chart
    a = cg.get_coin_ohlc_by_id(id=i, vs_currency=currency, days = Horizon)

    df = pd.DataFrame(data=a, columns = ['Date','Open','High', 'Low', 'Close'])

    fig2 = go.Figure(data=[go.Candlestick(x=df['Date'],open=df['Open'], high=df['High'],low=df['Low'], close=df['Close'])])
                          
    fig2.show()
    


In [ ]:
import numpy as np
import pandas as pd

frames=[]

for rows in single['id']:
  for i in single['prices']:
      
    df = pd.DataFrame(i,columns = ['date','price'], index = None)
    df['id'] = rows

  frames.append(df)

combined = pd.concat(frames)

tokens = len(combined['id'].unique())


path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

combined.to_csv(path+'/CoinGecko_prices.csv')

combined.info()

print('REPORT ')
print('Tokens total: '+ str(tokens))

In [ ]:
combined.head()

In [ ]:
combined.describe()